In [54]:
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch

In [3]:
dataset = pd.read_pickle('./MIMIC40/data_40_correct.pkl')

In [44]:
patient_feature = dataset[0]
label = dataset[1]
label_time = dataset[2]

In [ ]:
###Readin Data

In [118]:
tryme2 = np.ones((5,4))

In [95]:
tryme2

array([[ 1.,  1.,  0.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.]])

In [121]:
tryme2[[2:4, ],2] = 0

In [79]:
a = np.tril(np.ones((3,3))*(np.array([1,2,3]).reshape(3,1))).flatten()
a[np.nonzero(a)]

array([ 1.,  2.,  2.,  3.,  3.,  3.])

In [96]:
x = [[i]*i for i in [1,2,4]]

In [98]:
x = sum(x,[])

In [99]:
x

[1, 2, 2, 4, 4, 4, 4]

In [111]:
4%3

1

In [127]:
group = 2
unit = 14
mtrx = np.zeros((unit, unit))
for i in range(int(unit/group)):
    sequence = list(np.arange(i, unit/group))
    #print(sequence)
    sequence = [int(j) for j in sequence if (j - i)%x[i] == 0]
    ##print(sequence)
    for index in sequence:
        mtrx[index*group:(index+1)*group,i*group:(i+1)*group] = 1

In [128]:
mtrx

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
         0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0., 

In [134]:
### Helper function
def block_tri(group_size,scale, num_units, mode = 'original'):
    '''
    group_size: the size of each group
    num_units: group_size x scale
    mode: the way of connection, original, shift, fully connect
    return: tensor mask
    '''
    mtrx = np.zeros((num_units, num_units))
    if mode == 'original':
        for i in range(num_units/group_size - 1):
            mtrx[i*group_size: , i*group_size:(i+1)*group_size] = 1
    elif mode == 'shift': 
        mtrx = np.zeros((num_units, num_units))
        for i in range(int(num_units/group_size)):
            refer_li = sum([[i]*i for i in scale],[])
            length = refer_li[i]
            sequence = list(np.arange(i, num_units/group_size))
            #print(sequence)
            sequence = [int(j) for j in sequence if (j - i)%length == 0]
            ##print(sequence)
            for index in sequence:
                mtrx[index*group_size:(index+1)*group_size,i*group_size:(i+1)*group_size] = 1
    return mtrx 

#def activate_index()

In [136]:
block_tri(2,[1,2,4],14, mode = 'shift') == mtrx

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         

In [ ]:
torch.from_numpy(np.array()).type(torch.FloatTensor)

In [56]:
tryme = torch.Tensor([[1,2],[2,3]])

In [57]:
tryme


 1  2
 2  3
[torch.FloatTensor of size 2x2]

In [ ]:
### Model
#forward
#cell_class, step
class Clock_NN(nn.Module):
    def __init__(self, scale, group_size = 1, activation_fun = F.relu, mean = 0, std = 0.1, input_dim = 48)
        '''
        scale: the updating frequency, a list
        group_size: the number of nodes in each scale, default is 1.
        activation_function
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.scale = scale
        self.group_size = group_size
        if mode = 'orginal':
            self.num_units = self.scale*self.group_size
        elif mode = 'shift':
            self.num_units = sum(self.scale)*self.group_size
            
        self.class_dim = 2
        self.input_dim = input_dim
        self.linear_h = self.Linear(self.num_units*self.num_units)
        self.linear_o = self.Linear(self.num_units*self.class_dim)
        self.linear_i = self.Linear(self.num_units*self.input_dim)
        self.activation_fun = activation_fun
        self.mask = block_tri(self.group_size, self.scale, self.num_units)
        
        self.initial_weights(mean, std)
        
    def forward(self,#depends on what passed for model.train(), to be filled)
    
    def CW_RNN_Cell(self, x_input, hidden):
        '''
        x_input: number_feature x 1 vector, representing one time stamp
        hidden: output of the last cell
        '''
        hidden_next = self.linear_h(hidden)*self.mask*active(#TOBEFILLED) + self.linear_i(x_input)
        hidden_next = self.activation_fun(hidden_next)
        return hidden_next

    def init_hidden(self, bsz):
        h0 = Variable(torch.zeros(bsz, self.hidden_size))
        c0 = Variable(torch.zeros(bsz, self.hidden_size))
        return h0, c0
        
    def initial_weights(self, mean, std):
        lin_layers = [self.linear_h, self.linear]
        for layer in lin_layers:
            layer.weight.data.normal_(mean, std**2)
            layer.bias.data.fill_(0)
    

In [ ]:
### Training

In [ ]:
### Evaluation